In [ ]:
using KomaMRI, Plots, Unitful, MAT

seq = read_seq("Sequences/mpf_001_PhantomStudy_short.seq")

In [ ]:
plot_seq(seq; range=[0,40], height=400)

In [ ]:
function simulate_slice_profile_x(
    seq::Sequence; x=range(-2.e-2, 2.e-2, 200), sim_params=Dict{String,Any}("Δt_rf" => 1e-6)
)
    sim_params["return_type"] = "state"
    sys = Scanner()
    obj = Phantom{Float64}(; x=Array(x), z=zeros(size(x)))
    mag = simulate(obj, seq, sys; sim_params)
    return mag
end

In [ ]:
begin
    fa_degrees = Float64[]
    rf_indices = Int[]

    x = -0.01:0.0001:0.01
    Δx = x[2] - x[1]
    slice_thickness = 5e-3

    start_index = 3
    step_size = 2
    num_pulses = 1000

    for k in 0:num_pulses-1
        i = start_index + k * step_size  

        rf_event = seq[i]
        M = simulate_slice_profile_x(rf_event; x=x)
        Mxy = M.xy

        integral_Mxy = sum(Mxy) * Δx
        magnitude_integral = abs(integral_Mxy)

        α_rad = asin(clamp(magnitude_integral / slice_thickness, -1.0, 1.0))
        α_deg = α_rad * 180 / π

        push!(fa_degrees, α_deg)
        push!(rf_indices, i)
    end
end

In [ ]:
matfile = matopen("FApattern.mat")
FA = read(matfile, "FA")
close(matfile)

FA = vec(FA)

plot(fa_degrees,
    label = "Simulated",
    xlabel = "Pulse Number",
    ylabel = "Flip Angle (degrees)",
    linewidth = 2,
    grid = true,
    legend = :topright
)

plot!(FA,
    label = "Prescribed",
    linewidth = 2,
    linestyle = :dash
)